# Purpose

Use this notebook to present the outputs for the final output (or close to it). 

We need this notebook to show some clear examples on how to use the data and where/how we can use it.

# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
from datetime import datetime
import logging

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns

import mlflow

import subclu
from subclu.utils import set_working_directory
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.eda.aggregates import compare_raw_v_weighted_language
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl
)



print_lib_versions([np, pd, plotly, sns, subclu])

python		v 3.7.10
===
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
seaborn		v: 0.11.1
subclu		v: 0.3.1


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/mlflow/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas')

,experiment_id,name,artifact_location,lifecycle_stage
0,0,Default,./mlruns/0,active
1,1,fse_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/1,active
2,2,fse_vectorize_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/2,active
3,3,subreddit_description_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/3,active
4,4,fse_vectorize_v1.1,gs://i18n-subreddit-clustering/mlflow/mlruns/4,active
5,5,use_multilingual_v0.1_test,gs://i18n-subreddit-clustering/mlflow/mlruns/5,active
6,6,use_multilingual_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/6,active
7,7,use_multilingual_v1_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/7,active
8,8,use_multilingual_v1_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/8,active


# List & load artifacts from selected run

This is a run-ID for a process that outputs post-level and subreddit artifacts (after aggregation).

In [10]:
%%time
run_uuid = 'cf3c6dfb599b414a812085659e62ec85'

d_dfs_mlflow = mlf.read_run_artifact(
    run_id=run_uuid,
    artifact_folder='d_logged_dfs_subfolders/d_logged_dfs_subfolders.json',
    read_function='json',
)

list(d_dfs_mlflow.keys())

CPU times: user 313 ms, sys: 1.12 ms, total: 314 ms
Wall time: 472 ms


['df_post_level_agg_b_post_and_comments',
 'df_post_level_agg_c_post_comments_sub_desc',
 'df_sub_level_agg_a_post_only',
 'df_sub_level_agg_a_post_only_similarity',
 'df_sub_level_agg_a_post_only_similarity_pair',
 'df_sub_level_agg_b_post_and_comments',
 'df_sub_level_agg_b_post_and_comments_similarity',
 'df_sub_level_agg_b_post_and_comments_similarity_pair',
 'df_sub_level_agg_c_post_comments_and_sub_desc',
 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity',
 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity_pair']

In [11]:
f_post_embeddings = 'df_post_level_agg_c_post_comments_sub_desc'
# f_post_distance = 
# f_post_dist_pair =

f_sub_embeddings = 'df_sub_level_agg_c_post_comments_and_sub_desc'
f_sub_distance = 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity'
f_sub_dist_pair = 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity_pair'

## Load sub embeddings

In [12]:
%%time

df_sub_v = mlf.read_run_artifact(
    run_id=run_uuid,
    artifact_folder=f_sub_embeddings,
    read_function='pd_parquet',
)
print(df_sub_v.shape)

(628, 514)
CPU times: user 387 ms, sys: 101 ms, total: 487 ms
Wall time: 1.47 s


In [91]:
%%time

df_sub_d_pair = mlf.read_run_artifact(
    run_id=run_uuid,
    artifact_folder=f_sub_dist_pair,
    read_function='pd_parquet',
)
print(df_sub_d_pair.shape)

(12560, 11)
CPU times: user 231 ms, sys: 2.86 ms, total: 234 ms
Wall time: 651 ms


In [92]:
df_sub_d_pair.head()

,subreddit_id_a,subreddit_id_b,cosine_distance,subreddit_name_a,subreddit_name_b,German_posts_percent_a,German_posts_percent_b,manual_topic_and_rating_a,manual_topic_and_rating_b,post_median_word_count_a,post_median_word_count_b
0,t5_30jst,t5_2s82y,0.825948,1fcnuernberg,bundesliga,0.804878,0.343750,uncategorized,sports,10.0,14.0
1,t5_30jst,t5_2qt6s,0.821118,1fcnuernberg,hsv,0.804878,0.265625,uncategorized,uncategorized,10.0,14.5
2,t5_30jst,t5_2tndt,0.811017,1fcnuernberg,fussball,0.804878,0.814621,uncategorized,sports,10.0,11.0
3,t5_30jst,t5_2vz5t,0.793713,1fcnuernberg,effzeh,0.804878,0.189189,uncategorized,uncategorized,10.0,18.0
4,t5_30jst,t5_2t2g0,0.793503,1fcnuernberg,schalke04,0.804878,0.152778,uncategorized,uncategorized,10.0,12.0


# Load clusters

Currently ad-hoc

In [93]:
path_clusters_mod = 'manual_2021-07-20_15_13'
bucket_clusters = f'gs://i18n-subreddit-clustering/data/models/clustering/{path_clusters_mod}'

f_subs_meta_and_svd = 'df_subs_meta_and_svd-628_by_93.parquet'
f_subs_cluster_labels = 'df_subs_only-multiple_clustering_algos-628_by_58.parquet'

col_clustering = 'cluster_id_agg_ward_cosine_35'

In [94]:
!gsutil ls -r $bucket_clusters

gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/:
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_posts_and_subs-k_means_clustering-262854_by_48.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_posts_meta_and_svd-262226_by_72.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_subs_meta_and_svd-628_by_93.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_subs_only-multiple_clustering_algos-628_by_53.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_subs_only-multiple_clustering_algos-628_by_54.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_subs_only-multiple_clustering_algos-628_by_58.parquet
gs://i18n-subreddit-clustering/data/models/clustering/manual_2021-07-20_15_13/df_subs_only-multiple_clustering_algos-628_by_95.parquet
gs://i

In [95]:
%%time
df_subs_meta_plot = pd.read_parquet(f"{bucket_clusters}/{f_subs_cluster_labels}")

df_subs_meta_plot.shape

CPU times: user 47.8 ms, sys: 19.9 ms, total: 67.7 ms
Wall time: 881 ms


(628, 58)

In [96]:
# df_subs_meta_plot.head()

## Append subreddit label to pair-wise df

In [97]:
df_sub_d_pair = (
    df_sub_d_pair.merge(
        df_subs_meta_plot[['subreddit_name', col_clustering]].rename(columns={'subreddit_name': 'subreddit_name_b'}),
        how='left',
        on=['subreddit_name_b'],
    )
    .rename(columns={col_clustering: f"{col_clustering}_b"})
)

In [98]:
df_sub_d_pair.head()

,subreddit_id_a,subreddit_id_b,cosine_distance,subreddit_name_a,subreddit_name_b,German_posts_percent_a,German_posts_percent_b,manual_topic_and_rating_a,manual_topic_and_rating_b,post_median_word_count_a,post_median_word_count_b,cluster_id_agg_ward_cosine_35_b
0,t5_30jst,t5_2s82y,0.825948,1fcnuernberg,bundesliga,0.804878,0.343750,uncategorized,sports,10.0,14.0,18
1,t5_30jst,t5_2qt6s,0.821118,1fcnuernberg,hsv,0.804878,0.265625,uncategorized,uncategorized,10.0,14.5,18
2,t5_30jst,t5_2tndt,0.811017,1fcnuernberg,fussball,0.804878,0.814621,uncategorized,sports,10.0,11.0,18
3,t5_30jst,t5_2vz5t,0.793713,1fcnuernberg,effzeh,0.804878,0.189189,uncategorized,uncategorized,10.0,18.0,18
4,t5_30jst,t5_2t2g0,0.793503,1fcnuernberg,schalke04,0.804878,0.152778,uncategorized,uncategorized,10.0,12.0,18


# Examples

For a given subreddit, show:
- closest subreddits (pair)
- cluster assigned by Machine Learning model

In [102]:
sub_ = 'bundesliga'

# closest subs
display(
    style_df_numeric(
        df_sub_d_pair[df_sub_d_pair['subreddit_name_a'] == sub_]
        .drop(['subreddit_id_a', 'subreddit_id_b'], axis=1)
        .reset_index(drop=True)
        ,
        rename_cols_for_display=True,
        l_bar_simple=['cosine_distance', 'German_posts_percent_b'],
        float_round=4,
        
    )  #.hide_index()
    
)

# cluster assigned by ML
sub_id = df_subs_meta_plot.loc[df_subs_meta_plot['subreddit_name'] == sub_,
                               col_clustering].values[0]
display_items_for_cluster_id(sub_id)


# display overlap between closest subs and subs that are part of the selected cluster
# options:
#  - Could display in SVD or t-sne space as dots
#  - Could display as a table with flags for cluster IDs & distances
#. - or venn diagram?

,cosine distance,subreddit name a,subreddit name b,German posts percent a,German posts percent b,manual topic and rating a,manual topic and rating b,post median word count a,post median word count b,cluster id agg ward cosine 35 b
0,0.8976,bundesliga,fussball,34.38%,81.46%,sports,sports,14,11,18
1,0.8876,bundesliga,schalke04,34.38%,15.28%,sports,uncategorized,14,12,18
2,0.8822,bundesliga,fohlenelf,34.38%,7.83%,sports,uncategorized,14,14,18
3,0.8821,bundesliga,svw,34.38%,15.00%,sports,uncategorized,14,18,18
4,0.8693,bundesliga,vfbstuttgart,34.38%,11.21%,sports,uncategorized,14,11,18
5,0.8687,bundesliga,eintracht,34.38%,36.64%,sports,uncategorized,14,11,18
6,0.8593,bundesliga,hsv,34.38%,26.56%,sports,uncategorized,14,14,18
7,0.8259,bundesliga,1fcnuernberg,34.38%,80.49%,sports,uncategorized,14,10,18
8,0.8144,bundesliga,herthabsc,34.38%,24.39%,sports,sports,14,13,18
9,0.8034,bundesliga,effzeh,34.38%,18.92%,sports,uncategorized,14,18,18



Cluster ID: 18
  11 Subreddit count in group


manual_topic_and_rating,count,percent
uncategorized,8,72.7%
sports,3,27.3%


subreddit name,manual topic and rating,subreddit title,post median word count,German posts percent,English posts percent,image post type percent,text post type percent,users l28,posts l28,comments l28,rating,rating version,over 18
bundesliga,sports,German Bundesliga: News & Highlights,14,34.38%,58.68%,0.35%,35.76%,"33,860",168,"1,533",pg13,v1,None
fussball,sports,/r/fussball - Das deutschsprachige subreddit rund ums Thema Fußball,11,81.46%,12.01%,0.78%,31.59%,"15,044",291,739,None,None,None
herthabsc,sports,Hertha BSC,13,24.39%,65.85%,12.20%,36.59%,"4,891",22,129,pg,v3,None
schalke04,uncategorized,FC Schalke 04,12,15.28%,76.85%,17.59%,34.72%,"4,094",67,451,pg,v1,None
eintracht,uncategorized,Eintracht Frankfurt,11,36.64%,53.44%,25.19%,31.30%,"3,112",30,205,pg,v2,None
vfbstuttgart,uncategorized,VfB Stuttgart,11,11.21%,77.57%,14.95%,41.12%,"3,016",36,279,pg,v1,None
hsv,uncategorized,Hamburger SV - der Dino,14,26.56%,54.69%,12.50%,53.12%,"2,469",17,104,pg,v2,f
effzeh,uncategorized,Üvverall jitt et Fans vum FC Kölle,18,18.92%,79.28%,9.01%,43.24%,"2,351",48,382,pg,v1,None
1fcnuernberg,uncategorized,Wir sind der Club,10,80.49%,9.76%,19.51%,31.71%,"1,712",15,81,None,None,None
fohlenelf,uncategorized,r/fohlenelf: For Borussia fans on Reddit,14,7.83%,83.48%,12.17%,32.17%,"1,581",36,190,pg13,v2,None


'18'

# Loop through all clusters

In [64]:
col_manual_labels = 'manual_topic_and_rating'
cols_to_front = [
    'subreddit_name',
    
    'manual_topic_and_rating',
    
    'subreddit_title',
#     'subreddit_public_description',
    'post_median_word_count',
    
]
cols_to_exclude = [
    'subreddit_description',
    'subreddit_name_title_and_clean_descriptions',
    
    # cluster info not needed for now
    'cluster_id_kmeans20_all',
    'cluster_id_agg_ward_25',
    'cluster_id_agg_ward_35',
#     'cluster_id_agg_ward_cosine_35',
    'cluster_id_agg_avg_cosine_35',
    'cluster_id_agg_complete_cosine_35',
    'cluster_id_kmeans8',
    'cluster_id_kmeans15',
    'cluster_id_kmeans20',
    'cluster_id_hdbscan_cosine',
    'cluster_id_hdbscan_euclidean',
    'cluster_id_hdbscan_cosine_soft'
    
]


In [65]:
# [c for c in df_subs_meta_plot.columns if 'cluster_id' in c]

In [66]:
# cols_to_display = reorder_array(
#     cols_to_front,
#     df_subs_meta_plot.drop(cols_to_exclude, axis=1).columns
# )

cols_to_display = cols_to_front + [
#     'subreddit_name_title_and_clean_descriptions_word_count',
    
    'German_posts_percent',
    'English_posts_percent',
#     'other_language_posts_percent',
    
    'image_post_type_percent',
    'text_post_type_percent',
#     'link_post_type_percent',
#     'other_post_type_percent',


    'users_l28',
    'posts_l28',
    'comments_l28',
    
    'rating',
    'rating_version',
    'over_18',


]


In [67]:
# pd.set_option('display.max_colwidth', 180)
pd.set_option('display.width', 200)

In [85]:
def display_items_for_cluster_id(id_):
    """
    """
    mask_ = df_subs_meta_plot[col_clustering] == id_
    print(f"\nCluster ID: {id_}\n  {mask_.sum()} Subreddit count in group")
    

    display(
        value_counts_and_pcts(
            df_subs_meta_plot[mask_][col_manual_labels],
            add_col_prefix=False,
            reset_index=True,
            cumsum=False,
        ).hide_index()
    )
    display(
        style_df_numeric(
            df_subs_meta_plot[mask_][cols_to_display]
            .sort_values(by=['users_l28'], ascending=False)
            .head(15)
            ,
            rename_cols_for_display=True,
            l_bar_simple=[
                'German_posts_percent', 'English_posts_percent',
                'image_post_type_percent', 'text_post_type_percent',
                'users_l28',
            ]
        ).set_properties(subset=['subreddit title'], **{'width': '300px'}).hide_index()
    )
    

In [103]:
# cluster assigned by ML
for id_ in sorted(df_subs_meta_plot[col_clustering].unique()):
    display_items_for_cluster_id(id_)
#     mask_ = df_subs_meta_plot[col_clustering] == id_
#     print(f"\nCluster ID: {id_}\n  {mask_.sum()} Subreddit count in group")
    

#     display(
#         value_counts_and_pcts(
#             df_subs_meta_plot[mask_][col_manual_labels],
#             add_col_prefix=False,
#             reset_index=True,
#             cumsum=False,
#         ).hide_index()
#     )
#     display(
#         style_df_numeric(
#             df_subs_meta_plot[mask_][cols_to_display]
#             .sort_values(by=['users_l28'], ascending=False)
#             .head(10)
#             ,
#             rename_cols_for_display=True,
#             l_bar_simple=[
#                 'German_posts_percent', 'English_posts_percent',
#                 'image_post_type_percent', 'text_post_type_percent',
#                 'users_l28',
#             ]
#         ).set_properties(subset=['subreddit title'], **{'width': '300px'}).hide_index()
#     )


Cluster ID: 0
  11 Subreddit count in group


manual_topic_and_rating,count,percent
uncategorized,4,36.4%
food and drink,3,27.3%
reddit institutions,2,18.2%
outdoors and nature,1,9.1%
home and garden,1,9.1%


subreddit name,manual topic and rating,subreddit title,post median word count,German posts percent,English posts percent,image post type percent,text post type percent,users l28,posts l28,comments l28,rating,rating version,over 18
germanypics,outdoors and nature,Germany Pics,5,25.76%,60.61%,36.36%,0.00%,"5,554",70,101,pg,v3,None
spabiergang,uncategorized,Spabiergang,5,81.98%,0.90%,64.86%,8.11%,"4,881",48,271,None,None,None
bavaria,uncategorized,"Bayern, Bavaria, Baviera, Bavarska, Bavorsko, Βαυαρία, Bajorország, Bawaria, Бавария",7,50.00%,35.94%,35.16%,25.78%,"4,814",51,321,None,None,None
garten,home and garden,alles über den Garten,12,97.65%,0.88%,32.65%,26.47%,"3,837",131,713,pg,None,None
bier,food and drink,Bier,9,50.00%,29.41%,32.35%,17.65%,"1,966",24,115,r,None,None
naturfreunde,uncategorized,naturfreunde,6,91.04%,0.50%,46.27%,2.99%,"1,264",75,372,None,None,None
wasserknaben,uncategorized,WasserKnaben,5,93.33%,0.00%,20.00%,13.33%,885,9,35,None,None,None
daheim,reddit institutions,Denn Daheim ist immer am schönsten!,7,88.89%,11.11%,11.11%,11.11%,646,8,16,pg,None,None
augenbleiche,reddit institutions,r/Augenbleiche,5,100.00%,0.00%,33.33%,4.76%,303,15,9,None,None,None
einheitsgebot,food and drink,einheitsgebot,6,95.24%,0.00%,78.57%,4.76%,288,19,10,None,None,f



Cluster ID: 1
  13 Subreddit count in group


manual_topic_and_rating,count,percent
gaming,9,69.2%
"culture, entertainment, music",2,15.4%
reddit institutions,1,7.7%
uncategorized,1,7.7%


...

manual_topic_and_rating,count,percent
gaming,5,83.3%
uncategorized,1,16.7%


subreddit name,manual topic and rating,subreddit title,post median word count,German posts percent,English posts percent,image post type percent,text post type percent,users l28,posts l28,comments l28,rating,rating version,over 18
pcbaumeister,gaming,PCBaumeister,12,75.44%,15.79%,7.02%,29.82%,"10,466",75,363,None,None,None
nintendode,gaming,Nintendo Deutschland,8,58.06%,14.52%,14.52%,35.48%,"6,772",69,174,None,None,f
bapcsalesgermany,uncategorized,Build a PC Sales - Germany,28,28.57%,57.14%,0.00%,42.86%,"1,111",5,35,pg,v1,None
xboxde,gaming,Xbox auf Deutsch,10,86.36%,2.27%,11.36%,11.36%,226,20,61,None,None,None
playsi,gaming,PlayStation auf Deutsch,10,91.67%,4.17%,10.42%,6.25%,121,20,11,None,None,None
switchde,gaming,Nintendo Switch: Deutschland,15,71.43%,28.57%,0.00%,42.86%,22,0,0,None,None,None



Cluster ID: 9
  22 Subreddit count in group


manual_topic_and_rating,count,percent
uncategorized,15,68.2%
over18_nsfw,7,31.8%


subreddit name,manual topic and rating,subreddit title,post median word count,German posts percent,English posts percent,image post type percent,text post type percent,users l28,posts l28,comments l28,rating,rating version,over 18
nicolekopchak,over18_nsfw,Nicole Kopchak,2,0.00%,50.00%,40.00%,10.00%,"35,715",13,10,nc17,v3,f
palinarojinski,uncategorized,Palina Rojinski,3,54.71%,7.65%,31.76%,0.59%,"22,992",96,494,None,None,None
heidiklum,over18_nsfw,Heidi Klum,3,25.25%,27.27%,33.33%,14.14%,"19,398",34,47,nc17,v3,f
nicolescherzinger,over18_nsfw,Nicole Scherzinger,2,8.47%,42.37%,32.20%,1.69%,"15,918",31,47,nc17,v3,None
lenagercke,uncategorized,Lena Gercke,2,13.16%,31.58%,46.49%,0.00%,"11,994",47,94,None,None,None
laralofthot,uncategorized,LaraLoftHot,4,73.33%,8.89%,46.67%,22.22%,"8,385",29,118,None,None,None
lenalandrut,uncategorized,Lena Meyer-Landrut,1,11.69%,33.77%,46.75%,0.00%,"8,151",38,84,None,None,None
annyaurora,over18_nsfw,Anny Aurora,3,6.90%,65.52%,58.62%,0.00%,"7,501",16,10,None,None,t
sarah_lombardi_new,uncategorized,Sarah_Lombardi_New,3,43.37%,9.64%,66.27%,1.20%,"6,167",20,18,None,None,None
sophiathomalla,uncategorized,Sophia Thomalla,1,10.91%,23.64%,69.09%,0.00%,"4,431",28,22,None,None,None
